# MD5

## Wypełnianie wiadomości $M$

In [1]:
def padding(x):
    k = len(x)
    if len(x)%512 <= 447:
        x += "1"
        zeros_count = 512 - len(x)%512 - 64
        x += "0"*zeros_count
        zeros_count = 512 - 448 - len(bin(k)[2:])
        x += "0"*zeros_count
        x += bin(k)[2:]
        return x
    elif len(x)%512 > 447:
        x += "1"
        zeros_count = 512 - len(x)%512
        x += "0"*zeros_count
        x += "0"*448
        zeros_count = 512 - 448 - len(bin(k)[2:])
        x += "0"*zeros_count
        x += bin(k)[2:]
        return x

## Funkcje logiczne

In [2]:
def SS(x, y):
   for i in range(y):
       x = x[1:] + x[0] 
   return x

def XOR(a, b):
    return ''.join('0' if a[i] == b[i] else '1' for i in range(len(a)))

def AND(a, b):
    return ''.join('1' if a[i] == b[i] == "1" else '0' for i in range(len(a)))

def OR(a, b):
    return ''.join('1' if a[i] == '1' or b[i] == "1" else '0' for i in range(len(a)))

def NOT(x):
    return ''.join('1' if x[i] == '0' else '0' for i in range(len(x)))

## Funkcje wewnątrz rundy 
#### (dodawanie w $\mathbb{Z}_{2^{32}}$)

In [3]:
def F(B, C, D):
    return OR(AND(B, C), AND(NOT(B), D))
def G(B, C, D):
    return OR(AND(B, D), AND(C, NOT(D)))
def H(B, C, D):
    return XOR(XOR(B, C), D)
def I(B, C, D):
    return XOR(C, OR(B, NOT(D)))


# Adding in Z_2^32

def ADD(a, b):
    a = int(a, 2)
    b = int(b, 2)
    x = bin((a + b)%(2**32))[2:]
    x = x.zfill(32)
    return x


## Generowanie $M_j$

In [4]:
def split_512(x):
    M = []
    for m in range(len(x)//512):
        M.append(x[m*512:(m+1)*512])
    return M

def divblock(M):
    B = M[0:512]
    W_i = [B[32*i:32*i+32] for i in range(0,16)]
    B = W_i
    return B


## Wartości $K_i$ i $S_i$

In [5]:
import math

K = [bin(int(abs(math.sin(i+1)*(2**32))))[2:].zfill(32) for i in range(64)]

II = [[7, 12, 17, 22], 
      [5,  9, 14, 20],
      [4, 11, 16, 23],
      [6, 10, 15, 21]]

S = [II[i // 16 ][i % 4] for i in range(64)]

# A = 0x67452301
# B = 0x10325476
# C = 0x98badcfe
# D = 0xefcdab89
# 
# HH = [bin(A)[2:].zfill(32), bin(B)[2:].zfill(32), bin(C)[2:], bin(D)[2:]]

HH = [bin(int("01234567", 16))[2:].zfill(32), bin(int("89abcdef", 16))[2:].zfill(32), bin(int("fedcba98", 16))[2:].zfill(32), bin(int("76543210", 16))[2:].zfill(32)]

# MD5

In [6]:
def MD5(m):
    
# ASCII standard conversion

    M = ''.join(format(ord(char), '08b') for char in m)
    M = split_512(padding(M))
    
# Block [0-n]

    H_ = HH.copy()

    for n in range(len(M)):
        if n == 0:
            CV = H_.copy()

        Mi = divblock(M[n])

            
# CV = [a, b, c, d]

# Round [0-63]

        for i in range(64):
            if i < 16:
                T = ADD(SS(ADD(ADD(ADD(F(CV[1], CV[2], CV[3]), CV[0]), Mi[i]), K[i]), S[i]), CV[1])
                CV[0] = CV[3]
                CV[3] = CV[2]
                CV[2] = CV[1]
                CV[1] = T

            elif 16 <= i < 32: 
                T = ADD(SS(ADD(ADD(ADD(G(CV[1], CV[2], CV[3]), CV[0]), Mi[(5*i+1) % 16]), K[i]), S[i]), CV[1])
                CV[0] = CV[3]
                CV[3] = CV[2]
                CV[2] = CV[1]
                CV[1] = T

            elif 32 <= i < 48:
                T = ADD(SS(ADD(ADD(ADD(H(CV[1], CV[2], CV[3]), CV[0]), Mi[(3*i+5) % 16]), K[i]), S[i]), CV[1])
                CV[0] = CV[3]
                CV[3] = CV[2]
                CV[2] = CV[1]
                CV[1] = T
                
            elif 48 <= i < 64:
                T = ADD(SS(ADD(ADD(ADD(I(CV[1], CV[2], CV[3]), CV[0]), Mi[7*i % 16]), K[i]), S[i]), CV[1])
                CV[0] = CV[3]
                CV[3] = CV[2]
                CV[2] = CV[1]
                CV[1] = T
                    
        for i in range(len(CV)):
            CV[i] = ADD(CV[i], H_[i])
            
            
# Initializing the H value (a-d) for the next block

        H_ = CV.copy()

# Make hash from vector H = CV

    xhash = ''
    for i in H_:
        xhash += i
    return f"The MD5 hash of the data is: {hex(int(xhash, 2))[2:]}"

In [7]:
MD5("")

'The MD5 hash of the data is: 2520df5474d1b17877bf086504076c48'

In [8]:
import hashlib

# Create an MD5 hash object
md5_hash = hashlib.md5()

# Update the hash object with the data you want to hash
data = b""
md5_hash.update(data)

# Get the hexadecimal representation of the hash
hash_hex = md5_hash.hexdigest()

print(f"The MD5 hash of the data is: {hash_hex}")

The MD5 hash of the data is: d41d8cd98f00b204e9800998ecf8427e
